# Thermal Insulation Optimization using Neural Networks

This notebook demonstrates the optimization of thermal insulation design using neural networks as surrogate models. The goal is to minimize heat loss by optimizing the thickness of three insulation layers subject to constraints.

## Problem Formulation
**Objective**: Minimize heat loss Q = 1/x₁ + 1.5/x₂ + 2.0/x₃

**Design Variables:**
- x₁: Thickness of insulation layer 1
- x₂: Thickness of insulation layer 2  
- x₃: Thickness of insulation layer 3

**Constraints:**
- 5x₁ + 2.5x₂ + x₃ = 1.5 (material budget constraint)
- x₁, x₂, x₃ > 0 (physical feasibility)

## Approach
1. **Generate Training Data**: Create dataset satisfying the constraint equation
2. **Train Surrogate Model**: Use neural network to approximate the heat loss function
3. **Optimize Design**: Use gradient-based optimization to find minimum heat loss configuration

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt

# Heat loss
def heat_loss(x1, x2, x3):
    Q = 1 / x1 + 1.5 / x2  + 2.0 / x3
    return Q

## 1. Import Libraries and Define Heat Loss Function

Import necessary libraries and define the thermal model:

**Heat Loss Function:**
```
Q = 1/x₁ + 1.5/x₂ + 2.0/x₃
```

This represents heat transfer through three insulation layers in series, where:
- Each term represents thermal resistance of a layer
- Coefficients (1, 1.5, 2.0) represent different thermal conductivities
- Smaller thickness values lead to higher heat loss (inverse relationship)

In [ ]:
# Generate training dataset
n_row = 50
n_col = 50
n_samples = n_row * n_col

x1 = torch.zeros(n_samples, 1)
x2 = torch.zeros(n_samples, 1)
x3 = torch.zeros(n_samples, 1)

x1_min, x1_max = 0.05, 0.25

for i in range(n_samples):
    row = int(i / n_col)
    col = i - row * n_col
    x1[i] = row * (x1_max - x1_min) / n_row + x1_min
    
    x2_min = 0.05
    x2_max = 0.6 - 2 * x1[i] - 0.05
    x2[i] =  col * (x2_max - x2_min) / n_col + x2_min
    
    x3[i] = 1.5 - 5 * x1[i] - 2.5 * x2[i]

x3_min = min(x3)
x3_max = max(x3)

# Normalize inputs and outputs
def normalize(data, data_min, data_max):
    return (data - data_min) / (data_max - data_min)

def denormalize(data, data_min, data_max):
    return data * (data_max - data_min) + data_min

# Normalize data
x1_norm = normalize(x1, x1_min, x1_max)
x2_norm = normalize(x2, x2_min, x2_max)
x3_norm = normalize(x3, x3_min, x3_max)
Q = heat_loss(x1, x2, x3)
Q_min, Q_max = Q.min(), Q.max()  # Save for denormalization
Q_norm = normalize(Q, Q_min, Q_max)

# Combine normalized inputs into a single tensor
inputs_norm = torch.cat((x1_norm, x2_norm), dim=1)
outputs_norm = Q_norm.view(-1, 1)

## 2. Generate Training Dataset

Create a systematic dataset that satisfies the constraint equation:

**Dataset Generation Strategy:**
1. **Grid Sampling**: Create 50×50 = 2,500 data points
2. **Constraint Satisfaction**: 
   - Choose x₁ from [0.05, 0.25] (first design variable range)
   - For each x₁, determine feasible range for x₂ based on constraint
   - Calculate x₃ = 1.5 - 5x₁ - 2.5x₂ (from constraint equation)
3. **Data Normalization**: Scale all inputs and outputs to [0,1] range for better neural network training

**Constraint Handling:**
- Ensures x₂ > 0.05 (minimum thickness)
- Ensures x₃ > 0 (feasible solution)
- Results in a feasible design space covering all practical configurations

In [ ]:
# Define a neural network model
class SurrogateModel(nn.Module):
    def __init__(self):
        super(SurrogateModel, self).__init__()
        self.fc1 = nn.Linear(2, 20)
        self.fc2 = nn.Linear(20, 20)
        self.fc3 = nn.Linear(20, 1)
    
    def forward(self, x):
        x = torch.relu(self.fc1(x))
        x = torch.relu(self.fc2(x))
        x = self.fc3(x)
        return x

## 3. Define Surrogate Neural Network Model

Create a neural network to approximate the heat loss function:

**Architecture:**
- **Input Layer**: 2 neurons (x₁, x₂) - x₃ is dependent via constraint
- **Hidden Layer 1**: 20 neurons with ReLU activation
- **Hidden Layer 2**: 20 neurons with ReLU activation  
- **Output Layer**: 1 neuron (heat loss Q)

**Purpose:**
The neural network serves as a differentiable surrogate model for the heat loss function, enabling gradient-based optimization while satisfying constraints implicitly through the training data.

In [ ]:
# Initialize the model, loss function, and optimizer
model = SurrogateModel()
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.002)

# Train the model
n_epochs = 10000
loss_history = []

for epoch in range(n_epochs):
    model.train()
    optimizer.zero_grad()
    
    # Forward pass
    predictions = model(inputs_norm)
    loss = criterion(predictions, outputs_norm)
    loss.backward()
    optimizer.step()
    
    loss_history.append(loss.item())
    if (epoch + 1) % 200 == 0:
        print(f'Epoch [{epoch+1}/{n_epochs}], Loss: {loss.item():.5f}')

## 4. Train the Surrogate Model

Train the neural network to learn the relationship between layer thicknesses and heat loss:

**Training Configuration:**
- **Optimizer**: Adam with learning rate 0.002 (adaptive learning rate for better convergence)
- **Loss Function**: Mean Squared Error (MSE) for regression
- **Epochs**: 10,000 iterations
- **Monitoring**: Print loss every 200 epochs to track training progress

**Training Process:**
1. Forward pass: Predict heat loss from input thicknesses
2. Compute MSE between predictions and actual heat loss values  
3. Backpropagate gradients and update network weights
4. Track loss history for convergence analysis

In [ ]:
# Plot the training loss
plt.figure(figsize=(8, 5))
plt.plot(loss_history, label='Training Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training Loss Curve')
plt.legend()
plt.grid(True)
plt.show()

## 5. Visualize Training Progress

Plot the training loss curve to verify model convergence:

**Loss Curve Analysis:**
- **Decreasing Trend**: Indicates the model is learning the heat loss function
- **Convergence**: Loss should stabilize at a low value
- **Overfitting Check**: Smooth decrease without oscillations suggests good training

A well-trained surrogate model is essential for reliable optimization results.

In [ ]:
# Initial guess for optimization
x_opt = torch.tensor([0.5, 0.5], requires_grad=True)

# Optimizer for the optimization process
optimizer = optim.Adam([x_opt], lr=0.2)

# Optimization loop
n_iterations = 4000
for iteration in range(n_iterations):
    optimizer.zero_grad()
    
    # Predict normalized Q
    Q_pred_norm = model(x_opt)
    Q_pred = denormalize(Q_pred_norm, Q_min, Q_max)  # Denormalize for penalty calculation
    
    # Denormalize inputs
    x1_denorm = denormalize(x_opt[0], x1_min, x1_max)
    x2_denorm = denormalize(x_opt[1], x2_min, x2_max)
    
    # Loss for minimization
    loss = Q_pred
    loss.backward()
    optimizer.step()
    
    if (iteration + 1) % 500 == 0:
        print(f'Iteration [{iteration + 1}/{n_iterations}], Predicted Q: {Q_pred.item():.4f}, x_opt: {x_opt.data.numpy()}')

# Final optimized values (denormalized)
x1_opt = denormalize(x_opt[0].detach(), x1_min, x1_max)
x2_opt = denormalize(x_opt[1].detach(), x2_min, x2_max)

print(f'x1= {x1_opt.item():.2f}')
print(f'x2= {x2_opt.item():.2f}')
print(f'x3= {1.5 - 5 * x1_opt.item() - 2.5 * x2_opt.item():.2f}')

## 6. Optimize Thermal Insulation Design

Use the trained surrogate model to find the optimal layer thicknesses that minimize heat loss:

**Optimization Setup:**
- **Initial Guess**: Start with x₁ = x₂ = 0.5 (normalized coordinates)
- **Optimizer**: Adam with learning rate 0.2 (higher learning rate for faster convergence)
- **Objective**: Minimize predicted heat loss Q
- **Iterations**: 4,000 optimization steps

**Optimization Process:**
1. **Forward Pass**: Use surrogate model to predict heat loss
2. **Denormalization**: Convert normalized predictions to physical units for constraint checking
3. **Gradient Descent**: Update design variables to minimize heat loss
4. **Monitoring**: Track optimization progress every 500 iterations

**Final Results:**
- Optimal x₁, x₂ values (denormalized to physical units)
- Corresponding x₃ calculated from constraint equation
- Minimum achievable heat loss value